In [10]:
!pip install -q datasets


In [11]:
from datasets import load_dataset
import re

DATASET = load_dataset("ZSvedic/humor-chains")

DATASET = DATASET["train"].shuffle(seed=25).select(range(2000))

def transform_conv(example):
  conversation = example["conversations"]
  reformatted_segments = []
  i = 0
  while i < len(conversation):
      if conversation[i]["from"] == "human":
          human_text = conversation[i]["value"]

          # Check if GPT follows
          if i + 1 < len(conversation) and conversation[i+1]["from"] == "gpt":
              assistant_text = conversation[i+1]["value"]
              reformatted_segments.append(f"<s>[INST] {human_text} [/INST] {assistant_text} </s>")
              i += 2  # Skip assistant
          else:
              # Human only
              reformatted_segments.append(f"<s>[INST] {human_text} [/INST] </s>")
              i += 1
      else:
          i += 1  # Skip stray GPT if any

  return {"text": "".join(reformatted_segments)}

# Apply transformation
transformed_dataset = DATASET.map(transform_conv)

# Keep only the text column
transformed_dataset = transformed_dataset.select_columns(["text"])

# Check result
print(transformed_dataset[0]["text"])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<s>[INST] I was going to post a time traveling joke, but you guys didn’t like it. [/INST] Well I laughed the first time I read it. </s>


In [12]:
transformed_dataset.push_to_hub("humorchains-llama2-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/271 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/anuzb/humorchains-llama2-1k/commit/a51a6007bf970bff077a8b6183c8ee3bb39ff72a', commit_message='Upload dataset', commit_description='', oid='a51a6007bf970bff077a8b6183c8ee3bb39ff72a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/anuzb/humorchains-llama2-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='anuzb/humorchains-llama2-1k'), pr_revision=None, pr_num=None)